In [1]:
import torch
import torch.nn as nn
import pandas as pd

import numpy as np

import data_process as dp

In [2]:
# 读取训练数据
train_data = pd.read_csv('train_data.csv')

In [3]:
#随机抽样30%的数据作为测试集
test_data = train_data.sample(frac=0.3)
train_data = train_data.drop(test_data.index)

In [4]:
inputs, labels = dp.data_processing(train_data)

inputs_val, labels_val = dp.data_processing(test_data)

print(inputs.shape, labels.shape, inputs_val.shape, labels_val.shape)

print(inputs[0], labels[0])

# 查看训练数据的标签分布
print(labels.sum(dim=0))

torch.Size([3567, 63]) torch.Size([3567, 4]) torch.Size([1529, 63]) torch.Size([1529, 4])
tensor([-1.2074, -1.2074, -1.2074,  0.7167,  0.9913, -1.3273,  0.6212,  1.0340,
        -1.3599,  0.5287,  1.0866, -1.3659,  0.4685,  1.0836, -1.3593,  0.5567,
         0.7721, -1.2982,  0.4370,  0.8141, -1.3411,  0.3693,  0.8669, -1.3657,
         0.3178,  0.9021, -1.3779,  0.5375,  0.8004, -1.2474,  0.4525,  0.9764,
        -1.2963,  0.5055,  1.0349, -1.3032,  0.5533,  0.9957, -1.2942,  0.5222,
         0.8591, -1.2105,  0.4610,  1.0089, -1.2686,  0.5142,  1.0421, -1.2761,
         0.5469,  0.9922, -1.2640,  0.5206,  0.9164, -1.1837,  0.4861,  1.0243,
        -1.2353,  0.5265,  1.0502, -1.2483,  0.5609,  1.0334, -1.2428]) tensor([0., 0., 1., 0.])
tensor([1155., 1153.,  783.,  476.])


In [5]:

class MLP(nn.Module):
    def __init__(self, in_features, dropout):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, 4)
        )


    def forward(self, x):
        return self.net(x)


In [6]:
net = MLP(63, 0.1)
X = torch.rand(size=(1, 63))
for layer in net.net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Flatten output shape:	 torch.Size([1, 63])
Linear output shape:	 torch.Size([1, 256])
ReLU output shape:	 torch.Size([1, 256])
Dropout output shape:	 torch.Size([1, 256])
Linear output shape:	 torch.Size([1, 64])
ReLU output shape:	 torch.Size([1, 64])
Dropout output shape:	 torch.Size([1, 64])
Linear output shape:	 torch.Size([1, 32])
ReLU output shape:	 torch.Size([1, 32])
Dropout output shape:	 torch.Size([1, 32])
Linear output shape:	 torch.Size([1, 4])


In [7]:
# 创建 MLP 模型实例
model = MLP(63, 0.1)

# 定义损失函数和优化器
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001)

In [8]:
num_epochs = 20
num_samples = inputs.shape[0]


for epoch in range(num_epochs):
    train_loss = 0.0
    for i in range(0, num_samples):
        input = inputs[i]
        label = labels[i]

        input = input.unsqueeze(0)
        label = label.unsqueeze(0)

 
        # 前向传播
        model.train()
        output = model(input)

        l = loss(output, label)
        
        # 反向传播和优化
        optimizer.zero_grad()
        l.backward()
        optimizer.step()

        train_loss += l.item()

    train_loss /= num_samples

    model.eval()
    val_loss = loss(model(inputs_val), labels_val).item()

    # 打印每个 epoch 的损失
    print(f'Epoch {epoch+1}/{num_epochs}, train_Loss: {train_loss}, val_Loss: {val_loss}')
    
    if val_loss < 0.005:
        break
    




    # 保存模型
torch.save(model.state_dict(), 'model.pth')

Epoch 1/20, train_Loss: 0.16936088325615054, val_Loss: 0.12470340728759766
Epoch 2/20, train_Loss: 0.10810642313745572, val_Loss: 0.0738038569688797
Epoch 3/20, train_Loss: 0.07612814613030038, val_Loss: 0.050370436161756516
Epoch 4/20, train_Loss: 0.05588407867283088, val_Loss: 0.033312827348709106
Epoch 5/20, train_Loss: 0.04483807569047247, val_Loss: 0.026108205318450928
Epoch 6/20, train_Loss: 0.03779605912999074, val_Loss: 0.023823048919439316
Epoch 7/20, train_Loss: 0.032034813963077155, val_Loss: 0.017572956159710884
Epoch 8/20, train_Loss: 0.028126888453993235, val_Loss: 0.014735937118530273
Epoch 9/20, train_Loss: 0.02541509842953917, val_Loss: 0.013247337192296982
Epoch 10/20, train_Loss: 0.02418274971162556, val_Loss: 0.011935444548726082
Epoch 11/20, train_Loss: 0.02172529632682792, val_Loss: 0.01166659127920866
Epoch 12/20, train_Loss: 0.020358691994811117, val_Loss: 0.009427092038094997


In [9]:
with torch.no_grad():
    model.eval()
    pre_val = model(inputs_val)

head = 10

print(pre_val.shape)
print(pre_val[:head])
print(labels_val[:head])

torch.Size([1529, 4])
tensor([[-7.0702e-03,  2.2360e-02,  9.6853e-01, -2.5976e-03],
        [ 9.8179e-02,  8.9146e-01,  1.3833e-02, -1.6402e-02],
        [-2.4707e-02, -9.9662e-04, -5.9935e-02,  1.1979e+00],
        [ 8.2033e-03, -4.4314e-03,  9.7333e-01,  9.0828e-03],
        [-1.1073e-03,  1.0008e+00,  8.7661e-04, -1.5885e-02],
        [ 8.9780e-03,  9.5973e-01, -3.6084e-02,  3.0365e-02],
        [ 9.4902e-01,  2.0541e-02,  9.5379e-03,  1.7116e-03],
        [ 5.8488e-02,  9.5471e-01, -1.1534e-02, -1.9287e-03],
        [-4.2591e-02,  1.0494e+00,  8.5162e-03, -2.3226e-03],
        [-4.1277e-03,  1.9999e-02,  9.6693e-01, -3.0737e-03]])
tensor([[0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]])
